# Example 10 - ROI pipeline (CNN features)

In this notebook, we'll set up ROI pipeline:

(1) Apply Sv threshold, given a range, use this threshold to detect ROIs
(2) For each ROI, match with annotations, compute IoU, get labels for ROIs
(3) For each ROI, save as numpy array, in filename, add contextual information (depth, relative altitude, lat/lon)

In [1]:
import os
import glob
import numpy as np
import pandas as pd
import pickle

from src.read_echogram import EchogramReader
from src.detect_ROI import ROIDetector
from src.ROI_features_context import FeatureExtractor
from src.transform_annotations import AnnotationTransformer
from src.match_annotations import OverlapAnnotation
from src.crop_ROI import ROICropper

from multiprocessing import Pool

## Step 1. Basic setting

In this step, we'll set up filenames (raw, bot) dir, threshold range

In [2]:
# mode: train, val, test

In [3]:
mode = "train"

In [4]:
# empty npy
npy_dir = "npy/" + mode + "_new/"

In [ ]:
# load raw + bot paths
pkl_dir = "pkl/"
with open(pkl_dir + f"{mode}_li.pickle", "rb") as handle:
    data_li = pickle.load(handle)

In [ ]:
# figure dir
fig_dir = "figures/"

In [ ]:
# threshold range
threshold_li = [-54, -56, -58, -60, -62, -64, -66, -68, -70, -72, -74, -76, -78, -80] 
# freq selection
freq_li = [18, 38, 120, 200]
# kernel size
kernel_size = 3
# iou threshold
iou_threshold = 0.5

In [ ]:
for threshold in threshold_li:
    new_npy_dir = npy_dir + "threshold_" + str(threshold)
    if os.path.exists(new_npy_dir):
        files = glob.glob(new_npy_dir + '/*')
        for f in files:
            os.remove(f)
    else:
        os.mkdir(new_npy_dir)

In [ ]:
# annotations
with open(pkl_dir + 'annotations_dict_new_p4.pickle', 'rb') as handle:
    annotations_dict = pickle.load(handle)

## Step 2. ROI detection

In this step, we'll detect ROIs with each Sv threshold. 

In [ ]:
# for each Sv threshold
    # for each filename
        # detect ROI
        # for each ROI, crop / save npy + get features
        # for each ROI, overlap with annotations + get labels (change to IoU, 0.5 threshold)
        # save in different folders

In [ ]:
def roi_threshold(a):
    i, j = a # [raw_path, bot_path]
    # read echogram
    echogram = EchogramReader(i, j, freq_li)
    filename, Sv_npy, surface_idx, bottom_idx, time, depth, positions = echogram()
    # select annotations
    flag = False
    if filename in annotations_dict:
        annotations_idx, labels = annotations_dict[filename]
        flag = True  
    # Sv thresheld
    for threshold in threshold_li:
        # (1) detect ROIs
        roi = ROIDetector(filename, Sv_npy, surface_idx, bottom_idx, fig_dir, threshold, kernel_size)
        img_shape, contours = roi()
        # (2) get contextual features
        features = FeatureExtractor(filename, contours, Sv_npy, bottom_idx, time, depth, positions)
        contours_sel, contours_features = features() 
        # (3) overlap with annotations, get labels
        if flag == True:
            overlap = OverlapAnnotation(filename, img_shape, annotations_idx, labels, contours_sel, fig_dir)
            contours_labels = overlap.compute_iou(iou_threshold)
        else:
            contours_labels = [0] * len(contours_sel) # no annotations
        # (4) save npy
        new_npy_dir = npy_dir + "threshold_" + str(threshold) + '/'
        crop = ROICropper(filename, contours_sel, contours_features, contours_labels, Sv_npy, new_npy_dir)
        crop()
    return True

In [ ]:
pool = Pool(10)
res_li = pool.map(roi_threshold, data_li) # list of [raw_path, bot_path]

D20191113-T083150
D20191014-T003421
D20191109-T150801
D20191021-T204829
D20191109-T154858
D20191031-T045858
D20190930-T235233
D20191001-T204744
D20191003-T044219
D20190926-T091313
D20191111-T165715
D20191001-T233426
D20191003-T021135
D20191110-T190705
D20191017-T145540
D20191109-T134607
D20191104-T124305
D20191104-T193349
D20191022-T083746
D20191103-T230131
D20191021-T192503
D20191107-T192619
D20191101-T135856
D20190927-T010917
D20190926-T160853
D20190928-T164457
D20191024-T213721
D20191019-T200745
D20191104-T010446
D20191023-T212900
D20190928-T232513
D20191023-T122637
D20191104-T221808


In [ ]:
pool.close()
pool.join()

In [ ]:
# 'Unclassified regions': 1, 'krill_schools': 2, 'fish_school': 3, 'AH_School': 4
# 50 echograms - 1 day
# add break, early stopping! -> start: 9:15 pm
# annotations to cv2, fail, Umat for array, Use, try except instead!
# est. 6 hours